In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

directory = '/content/drive/MyDrive/AI surgical training'
files = os.listdir(directory)
print(files)

['results', ' IMG_1026.MOV', ' IMG_1011.MOV', ' IMG_1025.MOV', ' IMG_1027.MOV', ' IMG_1024.MOV', ' IMG_1023.MOV', ' IMG_1019.MOV', ' IMG_1020.MOV', ' IMG_1022.MOV', ' IMG_4292.mov', ' IMG_6790.MOV', ' IMG_6791.MOV', ' IMG_3259.MOV', ' IMG_4283.mov', 'hand_landmarker.task', 'yolov8x-seg.pt', 'output']


In [ ]:
%pip install opencv-python-headless mediapipe ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

# Import the necessary base options
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

class DrawingOptions:
    def __init__(self, margin=10, font_size=1, font_thickness=1, handedness_text_color=(88, 205, 54)):
        self.MARGIN = margin
        self.FONT_SIZE = font_size
        self.FONT_THICKNESS = font_thickness
        self.HANDEDNESS_TEXT_COLOR = handedness_text_color

class HandDetector:
    def __init__(self, model_path="hand_landmarker.task", running_mode=VisionRunningMode.VIDEO, num_hands=2, min_hand_presence_confidence=0.1, drawing_options: DrawingOptions = DrawingOptions()):
        self.model_path = model_path
        self.running_mode = running_mode
        self.num_hands = num_hands
        self.min_hand_presence_confidence = min_hand_presence_confidence
        self.drawing_options = drawing_options
        self.landmarker = self.create_landmarker()

    def create_landmarker(self):
        options = HandLandmarkerOptions(
            base_options=BaseOptions(model_asset_path=self.model_path),
            running_mode=self.running_mode,
            num_hands=self.num_hands,
            min_hand_presence_confidence=self.min_hand_presence_confidence
        )
        return HandLandmarker.create_from_options(options)

    def draw_result_on_image(self, rgb_image, detection_result):
        hand_landmarks_list = detection_result.hand_landmarks
        handedness_list = detection_result.handedness
        annotated_image = np.copy(rgb_image)

        for idx in range(len(hand_landmarks_list)):
            hand_landmarks = hand_landmarks_list[idx]
            handedness = handedness_list[idx]

            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend(
                [landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks]
            )
            mp.solutions.drawing_utils.draw_landmarks(
                annotated_image,
                hand_landmarks_proto,
                mp.solutions.hands.HAND_CONNECTIONS,
                mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                mp.solutions.drawing_styles.get_default_hand_connections_style(),
            )

            height, width, _ = annotated_image.shape
            x_coordinates = [landmark.x for landmark in hand_landmarks]
            y_coordinates = [landmark.y for landmark in hand_landmarks]
            text_x = int(min(x_coordinates) * width)
            text_y = int(min(y_coordinates) * height) - self.drawing_options.MARGIN

            cv2.putText(
                annotated_image,
                f"{handedness[0].category_name}",
                (text_x, text_y),
                cv2.FONT_HERSHEY_DUPLEX,
                self.drawing_options.FONT_SIZE,
                self.drawing_options.HANDEDNESS_TEXT_COLOR,
                self.drawing_options.FONT_THICKNESS,
                cv2.LINE_AA,
            )

        return annotated_image


class NeedleDriverDetector:
    def __init__(self, model_path="yolov8x-seg.pt", confidence=0.85):
        self._model = YOLO(model_path)
        self._class_names = self._model.names
        self._conf = confidence
        self._model.fuse()

    def predict_on_image(self, rgb_image):
        result = self._model(rgb_image, conf=self._conf)[0]

        cls = result.boxes.cls.cpu().numpy()
        probs = result.boxes.conf.cpu().numpy()
        boxes = result.boxes.xyxy.cpu().numpy()

        if result.masks is not None:
            masks = result.masks.data.cpu().numpy()
            masks = np.moveaxis(masks, 0, -1)
            masks = scale_image(masks, result.masks.orig_shape)
            masks = np.moveaxis(masks, -1, 0)
        else:
            masks = None

        class_name = "scissors"
        cls_index = [index for index, name in self._class_names.items() if name == class_name][0]
        cls_filter_index = np.argwhere(cls == cls_index).flatten()
        boxes = boxes[cls_filter_index]
        if masks is not None:
            masks = masks[cls_filter_index]
        cls = cls[cls_filter_index]
        probs = probs[cls_filter_index]

        return boxes, masks, cls, probs

    def draw_result_on_image(self, rgb_image, masks, color, alpha):
        if masks is not None:
            for mask in masks:
                colored_mask = np.zeros_like(rgb_image, dtype=np.uint8)
                colored_mask[mask > 0.5] = color
                rgb_image = cv2.addWeighted(rgb_image, 1 - alpha, colored_mask, alpha, 0)
        return rgb_image

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
from ultralytics import YOLO
from ultralytics.utils.ops import scale_image
from tqdm import tqdm  # For displaying progress bar

# Import the necessary base options
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# HandDetector and NeedleDriverDetector classes defined here (same as in your code)

class Detector:
    def __init__(self, hand_model_path, needle_driver_model_path):
        self.hand_model_path = hand_model_path
        self.needle_driver_model_path = needle_driver_model_path

    def detect_and_draw(self, video_path, save_path=None):
        hand_detector = HandDetector(model_path=self.hand_model_path)
        needle_driver_detector = NeedleDriverDetector(model_path=self.needle_driver_model_path)

        capture = cv2.VideoCapture(video_path)
        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_number = 0

        with tqdm(total=frame_count, desc=f"Processing {os.path.basename(video_path)}") as pbar:
            while capture.isOpened():
                ret, frame = capture.read()
                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Create MediaPipe Image from numpy array
                mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
                timestamp_ms = int(capture.get(cv2.CAP_PROP_POS_MSEC))

                # Hand detection with video mode
                hand_results = hand_detector.landmarker.detect_for_video(mp_image, timestamp_ms)
                img_hand = hand_detector.draw_result_on_image(rgb_frame, hand_results)

                # Needle driver detection
                boxes, masks, _, _ = needle_driver_detector.predict_on_image(rgb_frame)

                # Only draw and save if scissors are detected
                if len(boxes) > 0:
                    annotated_frame = needle_driver_detector.draw_result_on_image(img_hand, masks, (0, 255, 0), 0.5)

                    if save_path:
                        if not os.path.exists(save_path):
                            os.makedirs(save_path)
                        cv2.imwrite(os.path.join(save_path, f"{frame_number}.jpg"), cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

                frame_number += 1
                pbar.update(1)  # Update the progress bar

        capture.release()

    def process_videos_in_directory(self, directory, save_directory):
        # Automatically gather all video files in the directory
        video_files = [f for f in os.listdir(directory) if f.lower().endswith(('.mov', '.mp4', '.avi'))]

        for video_name in video_files:
            video_path = os.path.join(directory, video_name)
            save_path = os.path.join(save_directory, os.path.splitext(video_name)[0])

            # Check if the save path already exists
            if os.path.exists(save_path):
                print(f"Skipping {video_name} as it has already been processed.")
                continue

            print(f"Processing video: {video_name}")
            self.detect_and_draw(video_path, save_path=save_path)
            print(f"Finished processing video: {video_name}")

# Example usage
directory = '/content/drive/MyDrive/AI surgical training'
save_directory = os.path.join(directory, 'output')

hand_model_path = '/content/drive/MyDrive/AI surgical training/hand_landmarker.task'
needle_driver_model_path = '/content/drive/MyDrive/AI surgical training/yolov8x-seg.pt'

detector = Detector(hand_model_path, needle_driver_model_path)
detector.process_videos_in_directory(directory, save_directory)

Skipping  IMG_1026.MOV as it has already been processed.
Skipping  IMG_1011.MOV as it has already been processed.
Skipping  IMG_1025.MOV as it has already been processed.
Skipping  IMG_1027.MOV as it has already been processed.
Skipping  IMG_1024.MOV as it has already been processed.
Skipping  IMG_1023.MOV as it has already been processed.
Skipping  IMG_1019.MOV as it has already been processed.
Skipping  IMG_1020.MOV as it has already been processed.
Skipping  IMG_1022.MOV as it has already been processed.
Skipping  IMG_4292.mov as it has already been processed.
Processing video:  IMG_6790.MOV
YOLOv8x-seg summary (fused): 295 layers, 71,797,696 parameters, 0 gradients, 344.1 GFLOPs


Processing  IMG_6790.MOV:   0%|          | 0/14172 [00:00<?, ?it/s]


0: 384x640 1 person, 3278.5ms
Speed: 14.0ms preprocess, 3278.5ms inference, 38.3ms postprocess per image at shape (1, 3, 384, 640)


Processing  IMG_6790.MOV:   0%|          | 1/14172 [00:06<25:32:06,  6.49s/it]


0: 384x640 (no detections), 4031.7ms
Speed: 4.2ms preprocess, 4031.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


...

Processing  IMG_6790.MOV:   3%|▎         | 359/14172 [22:02<13:27:23,  3.51s/it]


0: 384x640 1 person, 4040.4ms
Speed: 5.3ms preprocess, 4040.4ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)


Processing  IMG_6790.MOV:   3%|▎         | 360/14172 [22:07<14:11:51,  3.70s/it]


0: 384x640 1 person, 5099.1ms
Speed: 4.4ms preprocess, 5099.1ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)


Processing  IMG_6790.MOV:   3%|▎         | 361/14172 [22:12<15:57:59,  4.16s/it]